# 优化与深度学习

## 优化与估计

尽管优化方法可以最小化深度学习中的损失函数值，但本质上优化方法达到的目标与深度学习的目标并不相同。
- 优化方法目标：训练集损失函数值
- 深度学习目标：测试集损失函数值（泛化性）

In [1]:
%matplotlib inline
import sys
sys.path.append('/home/kesci/input')
import d2lzh1981 as d2l
from mpl_toolkits import mplot3d # 三维画图
import numpy as np

In [2]:
def f(x): return x * np.cos(np.pi * x)
def g(x): return f(x) + 0.2 * np.cos(5 * np.pi * x)

d2l.set_figsize((5, 3))
x = np.arange(0.5, 1.5, 0.01)
fig_f, = d2l.plt.plot(x, f(x),label="train error")
fig_g, = d2l.plt.plot(x, g(x),'--', c='purple', label="test error")
fig_f.axes.annotate('empirical risk', (1.0, -1.2), (0.5, -1.1),arrowprops=dict(arrowstyle='->'))
fig_g.axes.annotate('expected risk', (1.1, -1.05), (0.95, -0.5),arrowprops=dict(arrowstyle='->'))
d2l.plt.xlabel('x')
d2l.plt.ylabel('risk')
d2l.plt.legend(loc="upper right")

<Figure size 360x216 with 1 Axes>

## 优化在深度学习中的挑战
1. 局部最小值
2. 鞍点
3. 梯度消失

### 局部最小值


$$
f(x) = x\cos \pi x
$$


In [4]:
def f(x):
    return x * np.cos(np.pi * x)

d2l.set_figsize((4.5, 2.5))
x = np.arange(-1.0, 2.0, 0.1)
fig,  = d2l.plt.plot(x, f(x))
fig.axes.annotate('local minimum', xy=(-0.3, -0.25), xytext=(-0.77, -1.0),
                  arrowprops=dict(arrowstyle='->'))
fig.axes.annotate('global minimum', xy=(1.1, -0.95), xytext=(0.6, 0.8),
                  arrowprops=dict(arrowstyle='->'))
d2l.plt.xlabel('x')
d2l.plt.ylabel('f(x)');

<Figure size 324x180 with 1 Axes>

### 鞍点
* 对多维变量,所有自变量一阶偏导数都为0的店，如果Hessian矩阵特征值均为正,则改点为局部极小值点,如果Hessian矩阵特征值都为负,则该点为局部最大值点,如果Hessian矩阵特征值有正有负的点,则该点为鞍点.

In [6]:
x = np.arange(-2.0, 2.0, 0.1)
fig, = d2l.plt.plot(x, x**3)
fig.axes.annotate('saddle point', xy=(0, -0.2), xytext=(-0.52, -5.0),
                  arrowprops=dict(arrowstyle='->'))
d2l.plt.xlabel('x')
d2l.plt.ylabel('f(x)');

<Figure size 324x180 with 1 Axes>


$$
A=\left[\begin{array}{cccc}{\frac{\partial^{2} f}{\partial x_{1}^{2}}} & {\frac{\partial^{2} f}{\partial x_{1} \partial x_{2}}} & {\cdots} & {\frac{\partial^{2} f}{\partial x_{1} \partial x_{n}}} \\ {\frac{\partial^{2} f}{\partial x_{2} \partial x_{1}}} & {\frac{\partial^{2} f}{\partial x_{2}^{2}}} & {\cdots} & {\frac{\partial^{2} f}{\partial x_{2} \partial x_{n}}} \\ {\vdots} & {\vdots} & {\ddots} & {\vdots} \\ {\frac{\partial^{2} f}{\partial x_{n} \partial x_{1}}} & {\frac{\partial^{2} f}{\partial x_{n} \partial x_{2}}} & {\cdots} & {\frac{\partial^{2} f}{\partial x_{n}^{2}}}\end{array}\right]
$$


e.g.

In [7]:
x, y = np.mgrid[-1: 1: 31j, -1: 1: 31j]
z = x**2 - y**2

d2l.set_figsize((6, 4))
ax = d2l.plt.figure().add_subplot(111, projection='3d')
ax.plot_wireframe(x, y, z, **{'rstride': 2, 'cstride': 2})
ax.plot([0], [0], [0], 'ro', markersize=10)
ticks = [-1,  0, 1]
d2l.plt.xticks(ticks)
d2l.plt.yticks(ticks)
ax.set_zticks(ticks)
d2l.plt.xlabel('x')
d2l.plt.ylabel('y');

<Figure size 432x288 with 1 Axes>

### 梯度消失

In [8]:
x = np.arange(-2.0, 5.0, 0.01)
fig, = d2l.plt.plot(x, np.tanh(x))
d2l.plt.xlabel('x')
d2l.plt.ylabel('f(x)')
fig.axes.annotate('vanishing gradient', (4, 1), (2, 0.0) ,arrowprops=dict(arrowstyle='->'))

Text(2, 0.0, 'vanishing gradient')

<Figure size 432x288 with 1 Axes>

# 凸性 （Convexity）

## 基础

### 集合

![Image Name](https://cdn.kesci.com/upload/image/q5p1yhqzm8.png?imageView2/0/w/640/h/640)
![Image Name](https://cdn.kesci.com/upload/image/q5p1xz9gvz.png?imageView2/0/w/640/h/640)
![Image Name](https://cdn.kesci.com/upload/image/q5p1yue9fu.png?imageView2/0/w/320/h/320)



### 函数


$$
\lambda f(x)+(1-\lambda) f\left(x^{\prime}\right) \geq f\left(\lambda x+(1-\lambda) x^{\prime}\right)
$$


In [9]:
def f(x):
    return 0.5 * x**2  # Convex

def g(x):
    return np.cos(np.pi * x)  # Nonconvex

def h(x):
    return np.exp(0.5 * x)  # Convex

x, segment = np.arange(-2, 2, 0.01), np.array([-1.5, 1])
d2l.use_svg_display()
_, axes = d2l.plt.subplots(1, 3, figsize=(9, 3))

for ax, func in zip(axes, [f, g, h]):
    ax.plot(x, func(x))
    ax.plot(segment, func(segment),'--', color="purple")
    # d2l.plt.plot([x, segment], [func(x), func(segment)], axes=ax)

<Figure size 648x216 with 3 Axes>

### Jensen 不等式


$$
\sum_{i} \alpha_{i} f\left(x_{i}\right) \geq f\left(\sum_{i} \alpha_{i} x_{i}\right) \text { and } E_{x}[f(x)] \geq f\left(E_{x}[x]\right)
$$


---

## 性质
1. 无局部极小值
2. 与凸集的关系
3. 二阶条件

###  无局部最小值

证明：假设存在 $x \in X$ 是局部最小值，则存在全局最小值 $x' \in X$, 使得 $f(x) > f(x')$, 则对 $\lambda \in(0,1]$:

$$
f(x)>\lambda f(x)+(1-\lambda) f(x^{\prime}) \geq f(\lambda x+(1-\lambda) x^{\prime})
$$


### 与凸集的关系

对于凸函数 $f(x)$，定义集合 $S_{b}:=\{x | x \in X \text { and } f(x) \leq b\}$，则集合 $S_b$ 为凸集

证明：对于点 $x,x' \in S_b$, 有 $f\left(\lambda x+(1-\lambda) x^{\prime}\right) \leq \lambda f(x)+(1-\lambda) f\left(x^{\prime}\right) \leq b$, 故 $\lambda x+(1-\lambda) x^{\prime} \in S_{b}$

$f(x, y)=0.5 x^{2}+\cos (2 \pi y)$

In [10]:
x, y = np.meshgrid(np.linspace(-1, 1, 101), np.linspace(-1, 1, 101),
                   indexing='ij')

z = x**2 + 0.5 * np.cos(2 * np.pi * y)

# Plot the 3D surface
d2l.set_figsize((6, 4))
ax = d2l.plt.figure().add_subplot(111, projection='3d')
ax.plot_wireframe(x, y, z, **{'rstride': 10, 'cstride': 10})
ax.contour(x, y, z, offset=-1)
ax.set_zlim(-1, 1.5)

# Adjust labels
for func in [d2l.plt.xticks, d2l.plt.yticks, ax.set_zticks]:
    func([-1, 0, 1])

<Figure size 432x288 with 1 Axes>

### 凸函数与二阶导数

$f^{''}(x) \ge 0 \Longleftrightarrow f(x)$ 是凸函数

**必要性 ($\Leftarrow$):**

对于凸函数：

$$
\frac{1}{2} f(x+\epsilon)+\frac{1}{2} f(x-\epsilon) \geq f\left(\frac{x+\epsilon}{2}+\frac{x-\epsilon}{2}\right)=f(x)
$$

故:

$$
f^{\prime \prime}(x)=\lim _{\varepsilon \rightarrow 0} \frac{\frac{f(x+\epsilon) - f(x)}{\epsilon}-\frac{f(x) - f(x-\epsilon)}{\epsilon}}{\epsilon}
$$


$$
f^{\prime \prime}(x)=\lim _{\varepsilon \rightarrow 0} \frac{f(x+\epsilon)+f(x-\epsilon)-2 f(x)}{\epsilon^{2}} \geq 0
$$


**充分性 ($\Rightarrow$):**

令 $a < x < b$ 为 $f(x)$ 上的三个点，由拉格朗日中值定理:

$$
\begin{array}{l}{f(x)-f(a)=(x-a) f^{\prime}(\alpha) \text { for some } \alpha \in[a, x] \text { and }} \\ {f(b)-f(x)=(b-x) f^{\prime}(\beta) \text { for some } \beta \in[x, b]}\end{array}
$$


根据单调性，有 $f^{\prime}(\beta) \geq f^{\prime}(\alpha)$, 故:

$$
\begin{aligned} f(b)-f(a) &=f(b)-f(x)+f(x)-f(a) \\ &=(b-x) f^{\prime}(\beta)+(x-a) f^{\prime}(\alpha) \\ & \geq(b-a) f^{\prime}(\alpha) \end{aligned}
$$

In [11]:
def f(x):
    return 0.5 * x**2

x = np.arange(-2, 2, 0.01)
axb, ab = np.array([-1.5, -0.5, 1]), np.array([-1.5, 1])

d2l.set_figsize((3.5, 2.5))
fig_x, = d2l.plt.plot(x, f(x))
fig_axb, = d2l.plt.plot(axb, f(axb), '-.',color="purple")
fig_ab, = d2l.plt.plot(ab, f(ab),'g-.')

fig_x.axes.annotate('a', (-1.5, f(-1.5)), (-1.5, 1.5),arrowprops=dict(arrowstyle='->'))
fig_x.axes.annotate('b', (1, f(1)), (1, 1.5),arrowprops=dict(arrowstyle='->'))
fig_x.axes.annotate('x', (-0.5, f(-0.5)), (-1.5, f(-0.5)),arrowprops=dict(arrowstyle='->'))

Text(-1.5, 0.125, 'x')

<Figure size 252x180 with 1 Axes>

## 限制条件


$$
\begin{array}{l}{\underset{\mathbf{x}}{\operatorname{minimize}} f(\mathbf{x})} \\ {\text { subject to } c_{i}(\mathbf{x}) \leq 0 \text { for all } i \in\{1, \ldots, N\}}\end{array}
$$


### 拉格朗日乘子法
[Boyd & Vandenberghe, 2004](https://d2l.ai/chapter_references/zreferences.html#boyd-vandenberghe-2004)


$$
L(\mathbf{x}, \alpha)=f(\mathbf{x})+\sum_{i} \alpha_{i} c_{i}(\mathbf{x}) \text { where } \alpha_{i} \geq 0
$$


### 惩罚项

欲使 $c_i(x) \leq 0$, 将项 $\alpha_ic_i(x)$ 加入目标函数，如多层感知机章节中的 $\frac{\lambda}{2} ||w||^2$

### 投影


$$
\operatorname{Proj}_{X}(\mathbf{x})=\underset{\mathbf{x}^{\prime} \in X}{\operatorname{argmin}}\left\|\mathbf{x}-\mathbf{x}^{\prime}\right\|_{2}
$$



![Image Name](https://cdn.kesci.com/upload/image/q5p241skoc.png?imageView2/0/w/640/h/640)


# 凸集与凸函数

* **凸集**: $n$维欧氏空间中的点集,如果点集中的任意两点的连线都包含在这个点集,则这个点集是凸集.
* **凸集分离定理**: 两个不相交的非空凸集,一定可以用一个超平面分离这两个凸集.
* **凸函数(convex function)**:对于函数$f$定义域内的任意两点$x,y$,如果有$$f(\theta x+(1-\theta y))\leq \theta f(x)+(1-\theta)f(y)$$则称$f$为凸函数.如果$-f$为凸函数,则称$f$为凹函数(concave function)
* 一个函数的epigraph定义为:$$epi f:=\{(x,z):x \in dom f,z\geq f(x)\}$$
* 函数$f(x)$为凸函数当且仅当$epi f$为凸集.
* **严格凸函数** :凸函数定义中的不等号为严格不等号.
* **强凸函数** :存在$m>0$使得$f(x)-\frac{m}{2}||x||^2$为凸函数,其中的$m$称为函数$f$的强凸系数.
* 如果函数$f$可导,且对于定义域内的所有$x,y$都有$$f(y)\geq f(x)+\triangledown f(x)^T(y-x)$$则$f$为凸函数.更进一步,如果存在正数$m$使得$$f(y)\geq f(x)+\triangledown f(x)^T(y-x)+\frac{m}{2}||x-y||^2$$则$f$为系数为$m$的强凸函数.
* **凸函数的Hessian矩阵** :对于一个多元函数$f(x_1,x_2,...,x_n)$,定义其Hessian矩阵为$$\triangledown ^2 f:=(D_{ij})_{1\leq i,j\leq d} , D_{ij}=\frac{\partial ^2f}{\partial x_i\partial x_j}$$梯度算子是一元函数的导数在多元函数的推广,Hessian矩阵是一元函数的二阶导在多元函数的推广.
* 如果函数$f$二阶可导,且其Hessian矩阵半正定,则$f$是凸函数.
* 如果函数$f$二阶可导,且$\triangledown ^2 f-mI$半正定(m>0),则$f$是系数为m的强凸函数.
凸函数的一些性质:
* 凸函数的局部最小值是全局最小值.    
* $x_0$为可导的凸函数$f$的全局最小值$\Longleftrightarrow \triangledown f(x_0)=0$.
* 琴生不等式及推广:
$$
f(\theta x+(1-\theta y))\leq \theta f(x)+(1-\theta)f(y)
\\
f(\sum_{i=1}^k \theta_iX_i))\leq \sum_{i=1}^k\theta_i f(x_i),\sum_{i=1}^k \theta_i=1
\\
f(\int_{S} p(x)xdx)\leq \int_S f(x)p(x)dx,\int_{S} p(x)dx=1
\\
f(\mathbb{E}[x])\leq \mathbb{E}[f(x)]$$
* 多个凸函数的非负线性组合还是凸函数.凸函数的逐点最大化还是凸函数.凸函数的最小化($g(x)=inf_{y\in \mathcal{C}}f(x,y),\mathcal{C}$是非空凸集)还是凸函数.

# 最优化问题
* 定义:最优化问题是指在一定约束条件下,求解一个目标函数的最小值(或最大值)的问题.其数学形式为$$\min_{x\in \mathcal{C}} f(x)   $$称$f$为目标函数,$x$为决策变量,$\mathcal{C}$为可行域.如果可行域为整个空间,则称这个问题为无约束优化问题.
* **线性规划(Linear programming)** 研究线性约束条件下线性目标函数的极致问题,常用解法有单纯型法等.如果目标函数为二次函数,则称为**二次规划(Quadratic Programming)** 问题.机器学习中的许多优化问题都属于二次规划,例如,最小二乘回归,支持向量机等.如果一个最优化问题的目标函数是凸函数,且可行域是凸集,则称这样的问题为**凸优化问题** ,线性规划和二次规划都是凸优化问题.
* 极值点的$\varepsilon -\delta$定义
* 极值点是局部最优的,不一定是全局最优的,但凸函数的极小值点也是最小值点.
## 无约束问题的最优性条件
* 一阶必要条件: 在极值点$x_0$可微的函数$f(x)$,满足$\triangledown f(x_0)=0$,(如果是一元函数,就是在该点的导数等于零).
 * 二阶必要条件: 在极值点$x_0$二次可微的函数$f(x)$,满足$\triangledown f(x_0)=0,$且$\triangledown ^2f(x_0)=0$半正定.
上述两个条件都是必要非充分条件.
* 二阶充分条件: 在点$x_0$二次可微的函数$f(x)$,如果满足$\triangledown f(x_0)=0,$且$\triangledown ^2f(x_0)=0$正定,则点$x_0$是局部极小值点.
* 一阶充要条件:$x_0$是$n$维实空间上的可微凸函数$f(x)$的极值点的充要条件是梯度$\triangledown f(x_0)=0,$.
* 

